<a id="top"></a>
# Custom Mosaics <img align="right" style="padding:10px" src="../images/odc_logo.png">

This notebook can be used to create custom Landsat cloud-filtered mosaics for any time period and location. The mosaics can be output as GeoTIFF products for analysis in external GIS tools. The following mosaics are possible: Median (midpoint of spectral data) and Max-NDVI (maximum vegetation respons). These mosaics are quite common and will be of value to a large number of users. 

Users should review the "Cloud Statistics" notebook for more information about the temporal and spatial sample. An understanding of the underlying data is important for creating a valid mosaic for further analyses. In many cases, cloud contamination will create poor mosaics. With a careful review of the cloud coverage over a given region and time period, it is possible to improve the mosaics and avoid false outputs. 

What does this Notebook do?

* [Setup the Environment](#environment_setup)
* [Choose Platforms and Products](#platforms_products)
* [Load and Clean Data from the Data Cube](#load_data)
* [Create Mosaics and View the Results](#create_mosaic)
* [Create GeoTIFF Output Products](#output_products)
* [Contributors, Heritage and References](#contributors_heritage)

**<a href="https://www.youtube.com/watch?v=xMhlowyP9xU" target="_blank">Click Here</a>** to watch a 'how to' video explaining the functionlity of this notebook.

This notebook uses external functions including both a standard set of DCAL utilities (see folder [DCAL_utils](../DCAL_utils)) and a code behind file containing some specalised functions that make this page work (see file [DCAL_Custom_Mosaics.py](DCAL_Custom_Mosaics.py)).

<i><p style="color:red"><b>Red Text:</b> Indicates code blocks where user inputs can be provided.</p></i>

<a id="environment_setup"></a>
## Environment Setup [&#9652;](#top)
* **Surpress Python Warnings:** These warnings include notes about deprecitation of Python modules, etc.
* **Import Standard Python Modules:** Import external and standard Python modules including `datacube` which is part of the ODC framework, as well as `xarray` which is a useful external module.
* **Import DCAL Utilities:** Import a set of utilities developed by the DCAL team to help automate some of the more routine tasks in the notebooks. These utilities are located in `/DCAL_utils`.
* **Initialize Data Cube and Data Cube API**

[Back to Top](#top)

In [ ]:
# Supress Python Warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import Standard Python Modules
import datacube
import sys
import xarray as xr  
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Import notebook code behind (as cb) contaning function definitions specific to this notebook
import DCAL_Custom_Mosaics as cb

# Import DCAL utilities containing function definitions used generally across DCAL
# Add DCAL_utils to the sys.path
sys.path.append('../DCAL_utils')

# Import Data Cube API.
import data_access_api as dc_api

# Import display map functions
from dc_display_map import display_map

# Import clean mask function.
from clean_mask import landsat_qa_clean_mask

# Import mosaic functions.
from dc_mosaic import create_median_mosaic, create_max_ndvi_mosaic

# Import RGB functions.
from dc_rgb import rgb

# Import import export functions.
from import_export import export_slice_to_geotiff

In [ ]:
# Initialize Data Cube API (dc_api) and Data Cube (dc)
api = dc_api.DataAccessApi(config = '/home/localuser/.datacube.conf')
dc = api.dc

<a id="platforms_products"></a>
## <span id="plat_prod">Choose Platforms and Products [&#9652;](#top)</span>
* **Select Product:** Specify the product you desire to use. Note you can see the complete list of products in this Cube in the introductory notebook (<a href="../Open%20Data%20Cube%20Application%20Library%20Notebooks.ipynb#products" target="_blank">click here</a>).
* **View Dataset Spatial and Temporal Extents:** Queries the Cube and displays the min/max latitude and longitude, and the start and end date and time.
* **Select Spatial and Temporal Extents:** Select the area of interest (AOI) and time range based on extents available.

[Back to Top](#top)

In [ ]:
# Select a Product and Platform
# Examples: ghana, kenya, tanzania, sierra_leone, senegal

# product = "ls7_ledaps_kenya"
# platform = "LANDSAT_7"

# product = "ls8_lasrc_kenya"
# platform = "LANDSAT_8"

# product = "ls7_ledaps_ghana"
# platform = "LANDSAT_7"

product = "ls8_lasrc_ghana"
platform = "LANDSAT_8"

In [ ]:
# Print extents of the Data Cube so we know what data is available
extents = api.get_full_dataset_extent(platform = platform, product = product)

latitude_extents = (min(extents['latitude'].values),max(extents['latitude'].values))
longitude_extents = (min(extents['longitude'].values),max(extents['longitude'].values))
time_extents = (min(extents['time'].values),max(extents['time'].values))

print(latitude_extents)
print(longitude_extents)
print(time_extents)

<p style="color:red";><b>Select Spatial and Temporal Extents:</b> Select the area of interest (AOI) and time range based on extents available.</p>

**HINT:** Keep your region small (less than 0.5 deg square) to avoid memory overload issues

In [ ]:
# Select an analysis region (Lat-Lon) within the extents listed above. 
# Select a time period (Min-Max) within the extents listed above (Year-Month-Day)

# Mombasa, Kenya
# latitude = (-4.1, -3.9) 
# longitude = (39.5, 39.7) 

# Mau Forest - Western Kenya
# latitude = (-0.13406, 0.21307)
# longitude = (35.28322, 35.56681)

# Tano-Offin Forest - Ghana
# latitude = (6.5814, 6.8978 ) 
# longitude = (-2.2955, -1.9395) 

# Mining Region near Obuasi, Ghana
latitude = (6.0985, 6.2675)
longitude = (-2.050, -1.8629)

# Time Period
time_extents = ('2019-01-01', '2019-03-31')

In [ ]:
# Render a map of the Area of Interest.
display_map(latitude,longitude)

<a id="load_data"></a>
## Load and Clean Data from the Data Cube [&#9652;](#top)
* **Load Data:** Load data based on selections above.
* **Display Summary:** Display a summary of dataset.

[Back to Top](#top)

In [ ]:
# Load Data into xarray Dataset.
landsat_dataset = dc.load(latitude = latitude,
                          longitude = longitude,
                          platform = platform,
                          time = time_extents,
                          product = product,
                          measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa'])

In [ ]:
# Displays the first few values of each data array to check the content
# Latitude and Longitude numbers = number of pixels in each dimension
# Time = number of time slices in the dataset
landsat_dataset

In [ ]:
#Create cloud mask and cleaned dataset.
cloud_mask = landsat_qa_clean_mask(landsat_dataset, platform=platform)
cleaned_dataset = landsat_dataset.where(cloud_mask)

<a id="create_mosaic"></a>
## <span id="mosaics">Create Mosaics and View the Results [&#9652;](#top)</span>

> **Median Mosaic**  
>  Masks clouds from imagery and use the median-valued cloud-free pixels in the time series.
>  More specifically, each band (e.g. red) of each pixel is assigned its median across time.
>  So this mosaic method generates values that are not in the dataset.
>
> **Max NDVI Mosaic**  
>  Masks clouds from imagery and use the Max NDVI across time for cloud-free pixels in the time series. 
>  The maximum NDVI will represent the highest amount of green vegetation 

[Back to Top](#top)

In [ ]:
# Create mosaics
median_composite = create_median_mosaic(cleaned_dataset, cloud_mask)
max_ndvi_composite = create_max_ndvi_mosaic(cleaned_dataset, cloud_mask.values)

In [ ]:
# Show two mosaics ... Median and Maximum NDVI

# RGB image options
# Standard RGB = 321 = Red, Green, Blue
# False Color = 543 = SWIR1, NIR, Red
# False Color (Landsat Mosaic) = 742 = SWIR2, NIR, Green

fig, ax = plt.subplots(1, 2, figsize=(16, 8))
median_rgb = median_composite[['swir2', 'nir', 'green']].to_array()
maxndvi_rgb = max_ndvi_composite[['swir2', 'nir', 'green']].to_array()
median_rgb.plot.imshow(ax=ax[0], vmin=0, vmax=5000)
maxndvi_rgb.plot.imshow(ax=ax[1], vmin=0, vmax=5000)
ax[0].set_title('Median Mosaic'), ax[0].xaxis.set_visible(False), ax[0].yaxis.set_visible(False)
ax[1].set_title('Max NDVI Mosaic'), ax[1].xaxis.set_visible(False), ax[1].yaxis.set_visible(False)
plt.show()

<a id="output_products"></a>
## <span id="export">Create GeoTIFF Output Products [&#9652;](#top)</span>
<p style="color:red";><b>Set Export File Name:</b> Change the name of the output file, or it will be overwritten for each run. (Remove comment below marks to export.)</p>

[Back to Top](#top)

In [ ]:
# Remove the comment tags (#) to export a GeoTIFF output product 
# export_slice_to_geotiff(median_composite, 'geotiffs/DEMO_median_composite.tif')
# export_slice_to_geotiff(max_ndvi_composite, 'geotiffs/DEMO_max_ndvi_composite.tif')

In [ ]:
# Remove the comment tag (#) below to view the contents of the output folder
# !ls -lah geotiffs/

<a id="contributors_heritage"></a>
## Contributors, Heritage and References [&#9652;](#top)
The notebooks liked here have been created by Brian Killough (NASA, CEOS Systems Engineering Office) in collaboration with a number of Open Data Cube (ODC) users from the Africian Regional Data Cube, Digital Earth Australia, and other ODC users globally.

GitHub repositories referenced/forked:

- https://github.com/ceos-seo/data_cube_utilities <i>A set of utilities maintained by the CEOS SEO team in support of the Notebooks being developed.</i>
- https://github.com/ceos-seo/data_cube_notebooks <i>A set of Notebooks maintained by the CEOS SEO team.</i>
- https://github.com/GeoscienceAustralia/dea-notebooks <i>Forked this repo to include / adapt the [Beginers_guide](https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Beginners_guide) Notebooks. Basis for adaptation was [George Dyke's fork](https://github.com/gamedaygeorge/dea-notebooks).</i>

<i><b>NOTE:</b> Digital Earth Africa has adapted this notebook to run on their platform. The code is <a href="https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/master/DCAL/DCAL_notebooks/DCAL_Custom_Mosaics.ipynb" target="_blank">available here</a>. Their implementation cleans up some of the rougher edges that remain in this notebook.</i>

Last updated May 2020.

[Back to top](#top)